In [1]:
!nvidia-smi

Tue Jul 11 08:06:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   31C    P0    42W / 163W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%load_ext autoreload
%autoreload 2

## Infer

In [11]:
workspace_dir = '/nfs/Workspace/TeethSeg'

import os
import sys
from pathlib import PurePath
sys.path.append(workspace_dir)

import pandas as pd

from monai.transforms import (
   Compose,
   AddChanneld,
   EnsureChannelFirstd,
   LoadImaged,
   Orientationd,
   SqueezeDimd,
   LoadImage
)

from data_utils.data_loader_utils import load_data_dict_json
from data_utils.teeth_dataset import get_data_dicts
from data_utils.utils import get_pid_by_file
from data_utils.visualization import show_img_lbl, show_img_lbl_pred, show_img_lbl_preds, show_img_lbl_preds_overlap

from expers.infer_utils import get_tune_model_dir, get_data_path, get_pred_path

# mask
model_name = 'vnet3d'
# model_name = 'old_cswinunetr_edit_projout_divided_96_3_drop0_0_01'
data_name = 'tooth'
# exp_name = 'exp_1_noscale_intensity_depth2_4_32_2_window_96_3_drop0_0_01'
exp_name = 'exp_5_noscale_intensity_roi96_data50'
# edge
# model_name = 'old_cswinunetr_edit_projout_divided_96_3'
# data_name = 'tooth'
# exp_name = 'exp_6_noscale_intensity_dept2_4_32_2_roi96_3_dropout0_0_01_data50_edge'

# center
# model_name = 'old_cswinunetr_edit_projout_divided_96_3' #  swinunetr
# data_name = 'tooth'
# exp_name = 'exp_8_noscale_intensity_dept2_4_32_2_roi96_3_dropout0_0_01_data50_center'

# model_name = 'unet3d' #  swinunetr
# data_name = 'tooth'
# exp_name = 'exp_4_noscale_intensity_data50'

# swinunetr exp_8_noscale_intensity_roi96_data50_nopretrain
# vnet3d exp_5_noscale_intensity_roi96_data50
# attention_unet exp_4_noscale_intensity_data50
# unet3d exp_4_noscale_intensity_data50

root_exp_dir = os.path.join(
    workspace_dir, 
    'exps',
    model_name,
    data_name,
    exp_name
)


data_dir = '/nfs/Workspace/dataset/teeth/data50'
data_json = os.path.join(data_dir, 'data_new.json')
# data_json = os.path.join(data_dir, 'data_center.json')
model_dir = os.path.join(root_exp_dir, 'models')
log_dir = os.path.join(root_exp_dir, 'logs')
eval_dir = os.path.join(root_exp_dir, 'evals')
infer_dir = os.path.join(root_exp_dir, 'infers')

best_checkpoint = os.path.join(model_dir, 'best_model.pth')
final_checkpoint = os.path.join(model_dir, 'final_model.pth')

print('best model:', best_checkpoint)
print(f'infer dir: {infer_dir}\n')

tr_ds, val_ds, tt_ds = load_data_dict_json(data_dir, data_json)
data_dict = tt_ds[1]

data_dict['pred'] = get_pred_path(root_exp_dir, exp_name, data_dict['image'])
img_pth = data_dict['image']
lbl_pth = data_dict['label']
print('\nimg pth:', img_pth)
print('lbl pth:',lbl_pth)
print('pred pth:',data_dict['pred'])

%cd /nfs/Workspace/TeethSeg/exps/{model_name}/{data_name}

best model: /nfs/Workspace/TeethSeg/exps/vnet3d/tooth/exp_5_noscale_intensity_roi96_data50/models/best_model.pth
infer dir: /nfs/Workspace/TeethSeg/exps/vnet3d/tooth/exp_5_noscale_intensity_roi96_data50/infers

load json from /nfs/Workspace/dataset/teeth/data50/data_new.json
train files (30): ['1000966359_20180113', '1000889125_20191101', '1001162439_20180110', '1000983254_20180904', '1001152328_20180112', '1001218388_20180109', '1000915187_20180115', '1001223657_20180112', '1001142392_20180511', '1001142392_20180110', '1001142392_20180116', '1000971031_20180112', '1001022839_20180110', '1001173165_20171205', '1000995722_20180112', '1000889125_20200421', '1001127112_20180109', '1001247962_20180115', '1001133637_20180110', '1001172283_20190622', '1001152328_20180910', '1001232054_20180109', '1001020384_20180114', '1000813648_20180116', '1001218388_20161024', '1001068663_20180116', '1001009635_20180116', '1001218388_20171102', '1000889125_20171016', '1000889125_20180521']
val files (10):

In [12]:
!PYTHONPATH={workspace_dir} /opt/conda/envs/TeethSeg/bin/python /nfs/Workspace/TeethSeg/expers/infer.py \
--model_name={model_name} \
--data_name={data_name} \
--data_dir={data_dir} \
--model_dir={model_dir} \
--infer_dir={infer_dir} \
--checkpoint={best_checkpoint} \
--img_pth={img_pth} \
--lbl_pth={lbl_pth} \
--out_channels=2 \
--scale_intensity_type='range' \
--a_min=218 \
--a_max=3187 \
--space_x=0.4 \
--space_y=0.4 \
--space_z=0.4 \
--roi_x=96 \
--roi_y=96 \
--roi_z=96 \
--infer_overlap=0.25

cuda is available
model: vnet3d
=> loaded checkpoint '/nfs/Workspace/TeethSeg/exps/vnet3d/tooth/exp_5_noscale_intensity_roi96_data50/models/best_model.pth' (epoch 2541) (bestacc 0.978345513343811) (early stop count 0)
infer data: {'image': '/nfs/Workspace/dataset/teeth/data50/image/1001261667_20180109.nii.gz', 'label': '/nfs/Workspace/dataset/teeth/data50/label/1001261667_20180109.nii.gz'}
<class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.

infer test time aug: 13.728981453925371 s
dice: 0.9468929
hd95: 1.4142135623730951
avg dice: 0.9468929
avg hd95: 1.4142135623730951

infer test original:
dice: 0.9468929
hd95: 1.4142135623730951
avg dice: 0.9468929
avg hd95: 1.4142135623730951
2023-06-27 12:05:52,385 INFO image_writer.py:194 - writing: /nfs/Workspace/TeethSeg/exps/vnet3d/tooth/exp_5_noscale_intensity_roi96_data50/infers/1001261667_20180109.nii.

In [ ]:
from monai.transforms import (
    LoadImaged
)
dx = {'label': '/nfs/Workspace/dataset/teeth/cindydata/label/1001486953_20180109.nii.gz'}
dd = LoadImaged(keys=['label'])(dx)
lbl_space = list(dd['label_meta_dict']['pixdim'])[1:4]

In [ ]:
lbl_space

## Eval label

In [ ]:
data_dir = '/nfs/Workspace/CardiacSeg/dataset/chgh/block_06'

# img_pth='/nfs/Workspace/CardiacSeg/dataset/chgh/block_06/pid_1000.nii.gz'
# lbl_pth='/nfs/Workspace/CardiacSeg/dataset/chgh/block_06/pid_1000_gt.nii.gz'
# pred_pth='/nfs/Workspace/CardiacSeg/dataset/chgh/block_06/pid_1000_unetcnx.nii.gz'

print(data_dir)

In [ ]:
# training
!PYTHONPATH=/nfs/Workspace/CardiacSeg /opt/conda/bin/python /nfs/Workspace/CardiacSeg/expers/eval_label.py \
--out_channels=2 \
--a_min=-42 \
--a_max=423 \
--space_x=0.4 \
--space_y=0.4 \
--space_z=0.4 \
--data_dir={data_dir}
# --img_pth={img_pth} \
# --lbl_pth={lbl_pth} \
# --pred_pth={pred_pth} \

# Summarize exp results

## summartize data dict and dice, save out dir

In [ ]:
import os
from pprint import pprint
import shutil

from expers.infer_utils import get_data_path, get_pred_path, get_tune_dir, get_eval_csv_path, get_dice_val


pid = 'pid_1000'
root_dir = '/nfs/Workspace/TeethSeg/exps'
exp_name = 't_1'
model_names = ['unet3d', 'attention_unet', 'cotr', 'unetr', 'swinunetr', 'unetcnx_x0']
exp_names = [exp_name for _ in model_names] 
root_exp_dirs = [os.path.join(root_dir, m, 'chgh') for m in model_names]

is_save_output = False
out_dir = './output'
out_infer_dir = os.path.join(out_dir, 'infers')
out_eval_dir = os.path.join(out_dir, 'evals')
os.makedirs(os.path.join(out_infer_dir), exist_ok=True)
os.makedirs(os.path.join(out_eval_dir), exist_ok=True)


data_dict = get_data_path(data_dir, pid)
dice_dict = {}
for exp_name, model_name, root_exp_dir in zip(exp_names, model_names, root_exp_dirs):
    key = f'{model_name}'
    
    # add pred path to data dict
    data_dict[key] = get_pred_path(root_exp_dir, exp_name, data_dict['image'])

    # read dice by pid
    tune_dir = get_tune_dir(os.path.join(root_exp_dir, 'tune_results', exp_name))
    csv_pth = get_eval_csv_path(tune_dir)
    dice_dict[key] = get_dice_val(csv_pth, pid)
    
    if is_save_output:
        # copy pred file
        shutil.copyfile(data_dict[key], os.path.join(out_infer_dir, f'{model_name}_{pid}.nii.gz'))

        # copy csv file
        shutil.copyfile(csv_pth, os.path.join(out_eval_dir, f'{model_name}_best_model_eval.csv'))


pprint(data_dict)
pprint(dice_dict)

## load out dir

In [ ]:
import os
from pprint import pprint
import shutil

from expers.infer_utils import get_data_path, get_pred_path, get_tune_dir, get_eval_csv_path, get_dice_val

out_dir = './output'
out_infer_dir = os.path.join(out_dir, 'infers')
out_eval_dir = os.path.join(out_dir, 'evals')


data_dict = get_data_path(data_dir, pid)
dice_dict = {}
model_names = ['unet3d', 'attention_unet', 'cotr', 'unetr', 'swinunetr', 'unetcnx_x0']
for model_name in model_names:
    pred_path = os.path.join(out_infer_dir, f'{model_name}_{pid}.nii.gz')
    data_dict[model_name] = pred_path
    
    csv_pth = os.path.join(out_eval_dir, f'{model_name}_best_model_eval.csv')
    dice_dict[model_name] = get_dice_val(csv_pth, pid)

pprint(data_dict)
pprint(dice_dict)

In [ ]:
# !rm -rf output

In [ ]:
# !zip -r output.zip output

# Show exp results

In [ ]:
keys = list(data_dict.keys())

loader = Compose([
    LoadImaged(keys=keys),
    EnsureChannelFirstd(keys=keys),
    Orientationd(keys=keys, axcodes='PLI'),
    SqueezeDimd(keys=keys)
])

data = loader(data_dict)

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams["figure.facecolor"] = 'none'

from expers.infer_utils import get_img_lbl_preds

num_classes = 2
slice_idxs = [200] #[100,150,200,250]
alpha = 0.5
axis_off = True
#num_img = len(data_dict)
#fig_size = (num_img*5, num_img*len(slice_idxs))
num_img = len(data_dict) - 2 
fig_size = (num_img*6, len(slice_idxs)*6)
show_img = False
show_lbl_dc = True

keys = dice_dict.keys()

pred_titles = [f'{key} (dice: {dice_dict[key]:.3f})'for key in keys]

imgs, lbls, preds = get_img_lbl_preds(data, keys, slice_idxs, mode='a')

# show_img_lbl_preds
show_img_lbl_preds_overlap(
    imgs,
    lbls,
    preds,
    pred_titles,
    slice_idxs,
    num_classes,
    axis_off,
    alpha,
    fig_size,
    show_img, 
    show_lbl_dc
)

In [ ]:
slice_idxs = [250] #[150,250,350]
fig_size = (num_img*7, len(slice_idxs)*5)

imgs, lbls, preds = get_img_lbl_preds(data, keys, slice_idxs, mode='s', is_trans=True)

#show_img_lbl_preds
show_img_lbl_preds_overlap(
    imgs,
    lbls,
    preds,
    pred_titles,
    slice_idxs,
    num_classes,
    axis_off,
    alpha,
    fig_size,
    show_img, 
    show_lbl_dc
)

In [ ]:
slice_idxs = [250] # [200,250,350]
fig_size = (num_img*7, len(slice_idxs)*5)

imgs, lbls, preds = get_img_lbl_preds(data, keys, slice_idxs, mode='c', is_trans=True)

#show_img_lbl_preds
show_img_lbl_preds_overlap(
    imgs,
    lbls,
    preds,
    pred_titles,
    slice_idxs,
    num_classes,
    axis_off,
    alpha,
    fig_size,
    show_img, 
    show_lbl_dc,
)